In [1]:
using QuantEcon
using Interpolations
using Distributions

"""
Job search with learning
 Authors : Qingyin Ma and John Stachurski

transition function of the state process:
 ln w_t = theta + epsilon_t, (epsilon_t) ~iid N(0, gam_eps)
"""

type Search_Problem
    bet::Float64  # the discount factor
    c0_tilde::Float64  # the unemployment compensation
    gam_eps::Float64  # the variance of the shock process (epsilon_t)
    sig::Float64  # the coefficient of relative risk aversion
    mu_min::Float64  # the minimum of the mu grid
    mu_max::Float64  # the maximum of the mu grid
    mu_size::Int  # the number of grid points for mu
    mu_grids::Vector{Float64}  # the grid points for mu
    gam_min::Float64  # the minimum of the gam grid
    gam_max::Float64  # the maximum of the gam grid
    gam_size::Int  # the number of grid points for gam
    gam_grids::Array{Float64}  # the grid points for gam
    w_min::Float64  # the minimum of the w grid
    w_max::Float64  # the maximum of the w grid
    w_size::Int  # the number of grid points for w
    w_grids::Array{Float64}  # the grid points for w
    mc_size::Int  # the number of Monte Carlo draws
    draws::Array{Float64}  # the Monte Carlo draws used for integration
    
    function Search_Problem(;bet=0.95, 
                            c0_tilde=0.6, 
                            gam_eps=1.0, 
                            sig=3.0,
                            mu_min=-10.0, 
                            mu_max=10.0, 
                            mu_size=200,
                            mu_grids=Array{Float64}(100),
                            gam_min=1e-4, 
                            gam_max=10.0, 
                            gam_size=200, 
                            gam_grids=Array{Float64}(100),
                            w_min=1e-4, 
                            w_max=10.0, 
                            w_size=200, 
                            w_grids=Array{Float64}(100),        
                            mc_size=1000, 
                            draws=Array{Float64}(100))
        
        # create (update) grid points for (mu, gam, w)
        mu_grids = linspace(mu_min, mu_max, mu_size)
        gam_grids = linspace(gam_min, gam_max, gam_size)
        w_grids = linspace(w_min, w_max, w_size)
        
        draws = randn(mc_size)  # create (update) Monte Carlo draws
        return new(bet, c0_tilde, gam_eps, sig, 
                   mu_min, mu_max, mu_size, mu_grids,
                   gam_min, gam_max, gam_size, gam_grids, 
                   w_min, w_max, w_size, w_grids,
                   mc_size, draws)
    end
end
    


# The CRRA utility function
function util_func(w, sigma)
    if sigma == 1.0
        uw = log.(w)
    else
        uw = w.^(1. - sigma) / (1. - sigma)
    end
    return uw
end



# The Bellman operator
function Bellman_operator(sp::Search_Problem, v)
    # simplify names
    bet, c0_tilde, gam_eps, sig = sp.bet, sp.c0_tilde, sp.gam_eps, sp.sig
    w_grids, mu_grids, gam_grids = sp.w_grids, sp.mu_grids, sp.gam_grids
    w_size, mu_size, gam_size = sp.w_size, sp.mu_size, sp.gam_size
    mc_size, draws = sp.mc_size, sp.draws
    
    # interpolate (and extrapolate) to obtain a function
    v_f = extrapolate(interpolate((w_grids, mu_grids, gam_grids), v,
                      Gridded(Linear())), Flat())
    
    c0 = util_func(c0_tilde, sig) # the flow continuation reward
    
    # create empty space to store T(v)
    new_v = Array{Float64}(w_size, mu_size, gam_size)
    
    for k = 1:w_size
        w = w_grids[k]
        
        for j = 1:gam_size
            gam = gam_grids[j]
            # Bayesian updating of gam. "gam_prime" is a scalar
            gam_prime = 1. / (1. / gam + 1. / gam_eps)
            
            for i = 1:mu_size
                mu = mu_grids[i]
                # MC sampling : ln(w') ; a vector of size (mc_size * 1)
                lnw_prime = mu + sqrt(gam + gam_eps) * draws
                # MC sampling : w' ; a vector of size (mc_size * 1)
                w_prime = exp.(lnw_prime)
                # MC sampling : mu' ; a vector of size (mc_size * 1)
                mu_prime = gam_prime * (mu / gam + lnw_prime / gam_eps)
                # MC sampling : v_f(w',mu',gam')
                integ = Array{Float64}(mc_size)
                for m = 1:mc_size
                    integ[m] = v_f[w_prime[m], mu_prime[m], gam_prime]
                end
                new_v[k, i, j] = max(util_func(w, sig) / (1 - bet),
                                     c0 + bet * mean(integ))
            end
        end
    end
    return new_v
end



# The Jovanovic operator
function Jovanovic_operator(sp::Search_Problem, psi)
    # simplify names
    bet, c0_tilde, gam_eps, sig = sp.bet, sp.c0_tilde, sp.gam_eps, sp.sig
    mu_grids, gam_grids = sp.mu_grids, sp.gam_grids
    mu_size, gam_size = sp.mu_size, sp.gam_size
    mc_size, draws = sp.mc_size, sp.draws
    
    # interpolate (and extrapolate) to obtain a function
    psi_f = extrapolate(interpolate((mu_grids, gam_grids), psi, 
                     Gridded(Linear())), Flat())
    
    c0 = util_func(c0_tilde, sig) # the flow continuation reward
    
     # create empty space to store Q (psi)
    new_psi = Array{Float64}(mu_size, gam_size)
    
    for j = 1:gam_size
        gam = gam_grids[j]
        # Bayesian updating of gam. "gam_prime" is a scalar
        gam_prime = 1. / (1. / gam + 1. / gam_eps)
        
        for i = 1:mu_size
            mu = mu_grids[i]
            # MC sampling : ln (w') ; a vector of size (mc_size * 1)
            lnw_prime = mu + sqrt(gam + gam_eps) * draws
            # MC sampling : mu'
            mu_prime = gam_prime * (mu / gam + lnw_prime / gam_eps)
            # MC sampling : u(w') / (1-beta)
            integ_1 = util_func(exp.(lnw_prime), sig) / (1 - bet)
            # MC sampling : psi_f(mu',gam')
            integ_2 = Array{Float64}(mc_size)
            for k = 1:mc_size
                integ_2[k] = psi_f[mu_prime[k], gam_prime]
            end
            # MC sampling : max{u(w')/(1-beta), psi_f(mu',gam')}
            integ = max.(integ_1, integ_2)
            new_psi[i, j] = c0 + bet * mean(integ)
        end
    end
    return new_psi
end  

Jovanovic_operator (generic function with 1 method)

In [2]:
# Computation time of CVI : different parameter values

sig_vals = [2., 3., 4., 5., 6.] # key parameter values
loops_cvi = 50  # number of iterations performed for each parameter case

# create an empty matrix to store the computation time of CVI 
time_cvi = Array{Float64}(loops_cvi, length(sig_vals))

for j=1:length(sig_vals)
    sp = Search_Problem(sig=sig_vals[j], w_size=100, mu_size=100, gam_size=100)
    psi_0 = ones(sp.mu_size, sp.gam_size)
    
    for i=1:loops_cvi
        tic() # start the clock
        psi_new = Jovanovic_operator(sp, psi_0)
        time_cvi[i,j] = toq()
        psi_0 = psi_new
    end
    println("Loop $j finished ... ")
end 
       
 
# key loops for which we want to calculate time elapsed (CVI)
keyloops_cvi = [10, 20, 50]   
# create a vector to store the time elapsed (CVI) for the selected key loops
time_cvi_keyloops = Array{Float64}(length(keyloops_cvi), length(sig_vals))

for i=1:length(keyloops_cvi)
    for j=1:length(sig_vals)
        time_cvi_keyloops[i,j] = sum(time_cvi[1:keyloops_cvi[i], j])
    end
end

row_1 = time_cvi_keyloops[1,:]  # time taken : loop 10, all key parameter values
row_2 = time_cvi_keyloops[2,:]  # time taken : loop 20, all key parameter values
row_3 = time_cvi_keyloops[3,:]  # time taken : loop 50, all key parameter values


println("")
println("----------------------------------------------------------------")
println(" Computation time CVI : different parameter values")
println("")
println(" Parameter : sig values = $sig_vals")
println("")
println(" Key loops : $keyloops_cvi")
println("")
println(" Time of CVI (column: sig value) : ")
println("")
println(" Loop 10 : $row_1")
println(" Loop 20 : $row_2")
println(" Loop 50 : $row_3")
println("")
println("----------------------------------------------------------------")
println("")

Loop 1 finished ... 
Loop 2 finished ... 
Loop 3 finished ... 
Loop 4 finished ... 
Loop 5 finished ... 

----------------------------------------------------------------
 Computation time CVI : different parameter values

 Parameter : sig values = [2.0,3.0,4.0,5.0,6.0]

 Key loops : [10,20,50]

 Time of CVI (column: sig value) : 

 Loop 10 : [25.6532,30.3619,30.513,30.6087,30.4523]
 Loop 20 : [48.951,60.7655,60.812,61.2985,60.9269]
 Loop 50 : [118.917,151.907,152.257,152.781,152.578]

----------------------------------------------------------------



In [4]:
# Computation time VFI : different parameter values

sig_vals = [2., 3., 4., 5., 6.] # key parameter values
loops_vfi = 50  # number of iterations performed for each parameter case

# create an empty matrix to store the computation time of VFI 
time_vfi = Array{Float64}(loops_vfi, length(sig_vals))

for j=1:length(sig_vals)
    sp = Search_Problem(sig=sig_vals[j], mu_size=100, gam_size=100, w_size=100)
    v_0 = ones(sp.w_size, sp.mu_size, sp.gam_size)
    
    for i=1:loops_vfi
        tic() # start the clock
        v_new = Bellman_operator(sp, v_0)
        time_vfi[i,j] = toq()  # calculate the time elapsed
        v_0 = v_new
    end
    println("Loop $j finished ... ")
end 
       

# key loops for which we want to calculate time elapsed (VFI)
keyloops_vfi = [10, 20, 50]   
# create a vector to store the time elapsed (VFI) for the selected key loops
time_vfi_keyloops = Array{Float64}(length(keyloops_vfi), length(sig_vals))

for i=1:length(keyloops_vfi)
    for j=1:length(sig_vals)
        time_vfi_keyloops[i,j] = sum(time_vfi[1:keyloops_vfi[i], j])
    end
end

row_1 = time_vfi_keyloops[1,:]  # time taken : loop 10, all key parameter values
row_2 = time_vfi_keyloops[2,:]  # time taken : loop 20, all key parameter values
row_3 = time_vfi_keyloops[3,:]  # time taken : loop 50, all key parameter values


println("")
println("----------------------------------------------------------------")
println(" Computation time VFI : different parameter values")
println("")
println(" Parameter : sig values = $sig_vals")
println("")
println(" Key loops : $keyloops_vfi")
println("")
println(" Time of VFI (column: sig value) : ")
println("")
println(" Loop 10 : $row_1")
println(" Loop 20 : $row_2")
println(" Loop 50 : $row_3")
println("")
println("----------------------------------------------------------------")
println("")

Loop 1 finished ... 
Loop 2 finished ... 
Loop 3 finished ... 
Loop 4 finished ... 
Loop 5 finished ... 

----------------------------------------------------------------
 Computation time VFI : different parameter values

 Parameter : sig values = [2.0,3.0,4.0,5.0,6.0]

 Key loops : [10,20,50]

 Time of VFI (column: sig value) : 

 Loop 10 : [2606.9,2615.06,2618.34,2606.51,2625.71]
 Loop 20 : [5206.84,5220.0,5226.87,5221.45,5242.67]
 Loop 50 : [13026.4,13066.7,13159.4,13099.6,13135.4]

----------------------------------------------------------------



In [2]:
# Computation time of CVI : different grid sizes

w_size_vals = [50, 300]
mu_size_vals = [50, 300]
gam_size_vals = [50, 300]

loops_cvi = 50  # number of iterations performed for each case

# create an empty matrix to store the computation time of CVI 
time_cvi = Array{Float64}(loops_cvi, length(mu_size_vals))

for j=1:length(mu_size_vals)
    sp = Search_Problem(sig=3., w_size=w_size_vals[j], mu_size=mu_size_vals[j], 
                        gam_size=gam_size_vals[j])
    psi_0 = ones(sp.mu_size, sp.gam_size)
    
    for i=1:loops_cvi
        tic() # start the clock
        psi_new = Jovanovic_operator(sp, psi_0)
        time_cvi[i,j] = toq()
        psi_0 = psi_new
    end
    println("Loop $j finished ... ")
end 
       
 
# key loops for which we want to calculate time elapsed (CVI)
keyloops_cvi = [10, 20, 50]   
# create a vector to store the time elapsed (CVI) for the selected key loops
time_cvi_keyloops = Array{Float64}(length(keyloops_cvi), length(mu_size_vals))

for i=1:length(keyloops_cvi)
    for j=1:length(mu_size_vals)
        time_cvi_keyloops[i,j] = sum(time_cvi[1:keyloops_cvi[i], j])
    end
end

row_1 = time_cvi_keyloops[1,:]  # time taken : loop 10, all key parameter values
row_2 = time_cvi_keyloops[2,:]  # time taken : loop 20, all key parameter values
row_3 = time_cvi_keyloops[3,:]  # time taken : loop 50, all key parameter values


println("")
println("----------------------------------------------------------------")
println(" Computation time CVI : different grid sizes")
println("")
println(" mu_sizes  : $mu_size_vals")
println(" gam_sizes : $gam_size_vals")
println("")
println(" Key loops : $keyloops_cvi")
println("")
println(" Time of CVI (column: grid sizes) : ")
println("")
println(" Loop 10 : $row_1")
println(" Loop 20 : $row_2")
println(" Loop 50 : $row_3")
println("")
println("----------------------------------------------------------------")
println("")

Loop 1 finished ... 
Loop 2 finished ... 

----------------------------------------------------------------
 Computation time CVI : different grid sizes

 mu_sizes  : [50,300]
 gam_sizes : [50,300]

 Key loops : [10,20,50]

 Time of CVI (row: key loops, column: grid sizes) : 

 [8.3968,291.702]
 [16.1743,576.944]
 [38.9979,1430.67]

----------------------------------------------------------------



In [2]:
# Computation time of VFI : different grid sizes

w_size_vals = [300]
mu_size_vals = [300]
gam_size_vals = [300]

loops_vfi = 50  # number of iterations performed for each parameter case

# create an empty matrix to store the computation time of CVI 
time_vfi = Array{Float64}(loops_vfi, length(mu_size_vals))

for j=1:length(mu_size_vals)
    sp = Search_Problem(sig=3., w_size=w_size_vals[j], mu_size=mu_size_vals[j], 
                        gam_size=gam_size_vals[j])
    v_0 = ones(sp.w_size, sp.mu_size, sp.gam_size)
    
    for i=1:loops_vfi
        tic() # start the clock
        v_new = Bellman_operator(sp, v_0)
        time_vfi[i,j] = toq()  # calulate the time elapsed
        v_0 = v_new
    end
    println("Loop $j finished ... ")
end 
       
 
# key loops for which we want to calculate time elapsed (VFI)
keyloops_vfi = [10, 20, 50]   
# create a vector to store the time elapsed (VFI) for the selected key loops
time_vfi_keyloops = Array{Float64}(length(keyloops_vfi), length(mu_size_vals))

for i=1:length(keyloops_vfi)
    for j=1:length(mu_size_vals)
        time_vfi_keyloops[i,j] = sum(time_vfi[1:keyloops_vfi[i], j])
    end
end

row_1 = time_vfi_keyloops[1,:]  # time taken : loop 10, all key parameter values
row_2 = time_vfi_keyloops[2,:]  # time taken : loop 20, all key parameter values
row_3 = time_vfi_keyloops[3,:]  # time taken : loop 50, all key parameter values


println("")
println("----------------------------------------------------------------")
println(" Computation time VFI : different grid sizes")
println("")
println(" w_sizes   : $w_size_vals")
println(" mu_sizes  : $mu_size_vals")
println(" gam_sizes : $gam_size_vals")
println("")
println(" Key loops : $keyloops_vfi")
println("")
println(" Time of VFI (column: grid sizes) : ")
println("")
println(" Loop 10 : $row_1")
println(" Loop 20 : $row_2")
println(" Loop 50 : $row_3")
println("")
println("----------------------------------------------------------------")
println("")

Loop 1 finished ... 

----------------------------------------------------------------
 Computation time VFI : different grid sizes

 w_sizes   : [300]
 mu_sizes  : [300]
 gam_sizes : [300]

 Key loops : [10,20,50]

 Time of VFI (column: grid sizes) : 

 Loop 10 : [76372.6]
 Loop 20 : [1.52349e5]
 Loop 50 : [3.8022e5]

----------------------------------------------------------------

